# Pytorch Training UI

### Import Libraries

In [1]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

In [2]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    py_file_location = "./drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch" # my private packages are stored here
    home_directory = './drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/' # my home directory is stored in ./LAB of google drive
    !pip install einops
else:
    py_file_location = './PrivatePackages/pytorch'
    home_directory = './'

sys.path.append(os.path.abspath(py_file_location))

from environment import *
from utils import *

from sklearn.model_selection import train_test_split

In [3]:
from model.model_class import LSTM, BERT, LSTM_DANN, BERT_DANN, LSTM_DBCE, BERT_DBCE

### Set Seed and Load Data

In [4]:
SEED = 2608

In [5]:
data1 = []
with open(home_directory + '/data/raw/comp90051-2024s1-project-1/domain1_train_data.json', 'r') as f:
    for line in f:
        data1.append(json.loads(line))

data2 = []
with open(home_directory + './data/raw/comp90051-2024s1-project-1/domain2_train_data.json', 'r') as f:
    for line in f:
        data2.append(json.loads(line))

data_test = []
with open(home_directory + '/data/raw/comp90051-2024s1-project-1/test_data.json', 'r') as f:
    for line in f:
        data_test.append(json.loads(line))

# create domain labels for data
for i in range(len(data1)):
    data1[i]['domain'] = 0
for i in range(len(data2)):
    data2[i]['domain'] = 1

In [6]:
# Train Val Test Split

# get labels for stratification
label1 = [instance['label'] for instance in data1]
label2 = [instance['label'] for instance in data2]

train_ix_1, val_test_ix_1 = train_test_split(range(len(data1)), test_size=0.3, random_state=SEED, stratify = label1)
train_ix_2, val_test_ix_2 = train_test_split(range(len(data2)), test_size=0.3, random_state=SEED, stratify = label2)
val_ix_1, test_ix_1 = train_test_split(val_test_ix_1, test_size=0.5, random_state=SEED, stratify = [data1[i]['label'] for i in val_test_ix_1])
val_ix_2, test_ix_2 = train_test_split(val_test_ix_2, test_size=0.5, random_state=SEED, stratify = [data2[i]['label'] for i in val_test_ix_2])

# split data according to the index from train_test_split
train_data_1 = [data1[i] for i in train_ix_1]
val_data_1 = [data1[i] for i in val_ix_1]
test_data_1 = [data1[i] for i in test_ix_1]
train_data_2 = [data2[i] for i in train_ix_2]
val_data_2 = [data2[i] for i in val_ix_2]
test_data_2 = [data2[i] for i in test_ix_2]

# combine the data
train_data = train_data_1 + train_data_2
val_data = val_data_1 + val_data_2
test_data = test_data_1 + test_data_2

---
### Models

#### 1. Prediction only

In [4]:
# LSTM
MAX_SENTENCE_LENGTH = 128
MIN_FREQUENCY = 0 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY) 
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class LSTM_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_recurrent_layers = 2
    bidirectional = False
    n_heads = 8
    dropout = 0.1
    n_mlp_layers = 1
    flatten = False
    activation = nn.ReLU()
    res_learning = True
    mask_flag = False # TODO
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 3
    loss = nn.BCELoss()
    # loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = nn.BCELoss()
    # validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    regularisation_loss = None
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory +  f'./results/'
    name = f'LSTM_Classifier'
    


model = LSTM(LSTM_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

NameError: name 'train_data' is not defined

In [ ]:
# BERT

MAX_SENTENCE_LENGTH = 512
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY) 
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 512 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 3 
    embedding_aggregation = 'cls' # TODO
    n_mlp_layers = 1
    res_learning = True
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 3
    loss = nn.BCELoss()
    # loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = nn.BCELoss()
    # validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'

model = BERT(BERT_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [7]:
# BERT

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY) 
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

# pretrain_x = pretrain_x[:100]
# pretrain_y = pretrain_y[:100]
# pretrain_mask = pretrain_mask[:100]
# pretrain_dom = pretrain_dom[:100]
# preval_x = preval_x[:100]
# preval_y = preval_y[:100]
# preval_mask = preval_mask[:100]
# preval_dom = preval_dom[:100]

print('---')

class BERT_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 512 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 1
    embedding_aggregation = 'cls' # TODO
    n_mlp_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 3
    loss = nn.BCELoss()
    # loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = nn.BCELoss()
    # validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'

model = BERT(BERT_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
pretrain_best_epoch = model.fit_pretrain(pretrain_x, pretrain_y, pretrain_dom, pretrain_mask, preval_x, preval_y, preval_dom, preval_mask)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval_pretrain(preval_x, preval_y, preval_dom, preval_mask, pretrain_best_epoch, evaluation_mode = True)

# # train the model (all cells except this one will print training log and evaluation at each batch)
# best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
# print()

# # as model automatically saves best epoch, will now load the best epoch and evaluate on test set
# model.load()
# model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
# model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

100%|██████████| 4000/4000 [00:00<00:00, 50987.76it/s]


class prior: 0.2222222222222222 0.7777777777777778
domain prior: 0.7222222222222222 0.2777777777777778
dom1 class prior: 0.5 0.5
dom2 class prior: 0.11538461538461539 0.8846153846153846


100%|██████████| 2700/2700 [00:00<00:00, 4748.50it/s] 


---


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 13/13 [00:02<00:00,  5.62it/s]
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


...

Epoch 5 Train | Loss:  8.2160 | Accuracy:  0.0000| Domain 1 Accuracy:  0.0000 | Domain 2 Accuracy:     nan| 


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 5 Val   | Loss:  8.2447 | Accuracy:  0.0200 | 
                            Domain 1 Accuracy:  0.0200 | Domain 2 Accuracy:     nan| 


 85%|████████▍ | 11/13 [00:02<00:00,  3.96it/s]


KeyboardInterrupt: 

In [15]:
import torch
import torch.nn as nn

# Define the predicted values (model outputs) and target values (ground truth labels)
predicted_logits = torch.tensor([[np.log(0.1), np.log(0.9)], [np.log(0.1), np.log(0.9)]])  # Example predicted logits
target_indices = torch.tensor([1, 0])  # Example target indices

# Instantiate the CrossEntropyLoss function
criterion_ce = nn.CrossEntropyLoss()

# Calculate the loss
loss_ce = criterion_ce(predicted_logits, target_indices)

print("Cross Entropy Loss:", loss_ce.item())


Cross Entropy Loss: 1.203972804325936


In [17]:
import torch
import torch.nn as nn

# Define the predicted values (model outputs) and target values (ground truth labels)
predicted_logits = torch.tensor([[np.log(0.1/0.9), np.log(0.9/0.1)], [np.log(0.1/0.9), np.log(0.9/0.1)]])  # Example predicted logits
target_indices = torch.tensor([1, 0])  # Example target indices

# Instantiate the CrossEntropyLoss function
criterion_ce = nn.CrossEntropyLoss()

# Calculate the loss
loss_ce = criterion_ce(predicted_logits, target_indices)

print("Cross Entropy Loss:", loss_ce.item())


Cross Entropy Loss: 2.2094946699280333


In [25]:
import torch
import torch.nn as nn

# Define the predicted values (model outputs) and target values (ground truth labels)
predicted_logits = torch.tensor([[0.1, 0.9], [0.1, 0.9]])  # Example predicted logits (2 instances, 2 classes)
target_indices = torch.tensor([1, 0])  # Example target indices

# Instantiate the CrossEntropyLoss function
criterion_ce = nn.CrossEntropyLoss()

# Calculate the loss
loss_ce = criterion_ce(predicted_logits, target_indices)

print("Cross Entropy Loss:", loss_ce.item())

Cross Entropy Loss: 0.771100640296936


In [ ]:
import torch
import torch.nn as nn

# Define the predicted values (model outputs) and target values (ground truth labels)
predicted_logits = torch.tensor([[np.log(0.1), np.log(0.9)], [np.log(0.1), np.log(0.9)]])  # Example predicted logits
target_indices = torch.tensor([1, 0])  # Example target indices

# Instantiate the CrossEntropyLoss function
criterion_ce = nn.CrossEntropyLoss()

# Calculate the loss
loss_ce = criterion_ce(predicted_logits, target_indices)

print("Cross Entropy Loss:", loss_ce.item())


In [16]:
import torch
import torch.nn as nn

# Define the predicted values (model outputs) and target values (ground truth labels)
predicted = torch.tensor([[0.1, 0.9], [0.1, 0.9]])  # Example predicted values after softmax
target = torch.tensor([[0, 1], [1, 0]])            # Example target values

# Instantiate the BCELoss function
criterion_bce = nn.BCELoss()

# Calculate the loss
loss_bce = criterion_bce(predicted, target.float())  # Ensure target is of float type

print("Binary Cross Entropy Loss:", loss_bce.item())


Binary Cross Entropy Loss: 1.2039728164672852


In [26]:
import torch
import torch.nn as nn

# Define the predicted values (model outputs) and target values (ground truth labels)
predicted = torch.tensor([0.9, 0.9])  # Example predicted values after softmax
target = torch.tensor([1, 0])            # Example target values

# Instantiate the BCELoss function
criterion_bce = nn.BCELoss()

# Calculate the loss
loss_bce = criterion_bce(predicted, target.float())  # Ensure target is of float type

print("Binary Cross Entropy Loss:", loss_bce.item())


Binary Cross Entropy Loss: 1.2039726972579956


---
# DANN

In [ ]:
# LSTM_DANN

MAX_SENTENCE_LENGTH = 512
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY) 
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class LSTM_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 512
    n_recurrent_layers = 2
    bidirectional = False
    n_heads = 8
    dropout = 0.1
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    flatten = False
    activation = nn.ReLU()
    res_learning = True
    mask_flag = False # TODO
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 32
    lr = 1e-5
    patience = 3
    loss = nn.BCELoss()
    # loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = nn.BCELoss()
    # validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    regularisation_loss = None
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory +  f'./results/'
    name = f'LSTM_DANN'

model = LSTM_DANN(LSTM_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
# BERT

MAX_SENTENCE_LENGTH = 512
MIN_FREQUENCY = 0 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 512 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 3
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 3
    loss = nn.BCELoss()
    # loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = nn.BCELoss()
    # validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_DANN'

model = BERT_DANN(BERT_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

---
# DBCE

In [ ]:
# BERT DBCE

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DBCE_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 64
    d_ff = 64 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 1
    embedding_aggregation = 'cls' # TODO
    n_mlp_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # domain_1_loss = nn.BCELoss()
    domain_1_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # domain_1_validation_loss = nn.BCELoss()
    domain_1_validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # domain_2_loss = nn.BCELoss()
    domain_2_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # domain_2_validation_loss = nn.BCELoss()
    domain_2_validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_prior = [pos_dom_prior, neg_dom_prior]
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'

model = BERT_DBCE(BERT_DBCE_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

---
# Evaluation and Inference

In [ ]:
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
EXPERIMENT_NAME = '2lstm_unidir_512d_8_512t40_wbce'

In [ ]:
future_pred_y = model.predict(future_x)

future_pred_y = [1 if x[1] > x[0] else 0 for x in future_pred_y]

predictions = pd.DataFrame({'id': range(len(future_pred_y)), 'class': future_pred_y})
predictions.to_csv(home_directory + f'predictions/{EXPERIMENT_NAME}_classification.csv', index=False)